# Regions investigation and workspace
This notebook allows to both:
- investigate the existing regioner on the given dataste
- develop new regioner and see the results live


In [1]:
from utils import * 
from sep.inspect import regions

## Prepare dataset

In [10]:
import sep.loaders.images
data_root = pick_path([root_dir / "tests" / "input"])
dataset = data_root / "humans"
data_loader = sep.loaders.images.ImagesLoader(str(dataset))

In [11]:
data_loader[0]

TypeError: 'ImagesLoader' object does not support indexing

## Investigate the chosen regioner

## Develop new regioner